# Modelos Lineales

Los modelos lineales son una clase de modelos que se utilizan ampliamente en la practica y se han estudiado ampliamente en las ultimas decadas. Los modelos lineales hacen una prediccion utilizando una funcion lineal de las caracteristicas de entrada.

## Modelos lineales para la regresion

Para la regresion, la formula de prediccion general para un modelo lineal es la siguiente:

ŷ = w[0] * x[0] + w[1] * x[1] + ... + w[p] * x[p] + b

En esta formula desde x[0] hasta x[p] son las caracteristicas de 1 solo punto de datos mientras que w y b son los parametros del modelo que se aprenden, y ŷ es la prediccion que hara el modelo.